In [1]:
%matplotlib inline                         
from pylab import *
from IPython import display
import time
from numpy.random import uniform
from scipy.integrate import odeint

m=500                                                           # Set Parameters
L=10
I=(1/12)*m*L**2


def Accelx(theta,FL,FR):
    a = -(FL*sin(theta) - FR*sin(theta))/m                      #Acceleration in X
    
    return a


def Accely(theta,FL,FR):                                        #Acceleration in Y
    a = -9.8 + (FL*cos(theta)+FR*cos(theta))/m
    
    return a

def angAccel(FL,FR):                                            #Alpha
    a = (FR*L*.5 - FL*L*.5)/I
    
    return a


def RKStep2Ddrone(iv,dt):                 ### Implement a single step of the Runge-Kutta method...
    P=zeros((4,2))
    x=iv[0,0]
    y=iv[0,1]
    Vx=iv[1,0]                            # Recieve Old Values            
    Vy=iv[1,1]
    theta=iv[2,0]
    Omega=iv[2,1]
    FL=iv[3,0]
    FR=iv[3,1]

    dVx1=dt*Accelx(theta,FL,FR)                          # 4 mini time steps
    dVy1=dt*Accely(theta,FL,FR)
    dOmega1=dt*angAccel(FL,FR)
    
    dx1=dt*Vx
    dy1=dt*Vy
    dtheta1=dt*Omega
    
    dVx2=dt*Accelx((theta + dtheta1/2),FL,FR)
    dVy2=dt*Accely((theta + dtheta1/2),FL,FR)
    dOmega2=dt*angAccel(FL,FR)
    
    dx2=dt*(Vx+(dVx1/2))
    dy2=dt*(Vy+(dVy1/2))
    dtheta2=dt*(Omega+(dOmega1/2))
    
    dVx3=dt*Accelx((theta + dtheta2/2),FL,FR)
    dVy3=dt*Accely((theta + dtheta2/2),FL,FR)
    dOmega3=dt*angAccel(FL,FR)
    
    dx3=dt*(Vx+(dVx2/2))
    dy3=dt*(Vy+(dVy2/2))
    dtheta3=dt*(Omega+(dOmega2/2))
    
    dVx4=dt*Accelx((theta + dtheta3),FL,FR)
    dVy4=dt*Accely((theta + dtheta3) ,FL,FR)
    dOmega4=dt*angAccel(FL,FR)
    
    dx4=dt*(Vx+dVx3)
    dy4=dt*(Vy+dVy3)
    dtheta4=dt*(Omega+dOmega3)
    
    Vxnew = Vx +(1.0/6)*(dVx1+dVx4+2*(dVx2+dVx3))                          # Weighted mini step average
    Vynew = Vy +(1.0/6)*(dVy1+dVy4+2*(dVy2+dVy3))
    newOmega = Omega +(1.0/6)*(dOmega1+dOmega4+2*(dOmega2+dOmega3))
    
    xnew = x +(1.0/6)*(dx1+dx4+2*(dx2+dx3))
    ynew = y +(1.0/6)*(dy1+dy4+2*(dy2+dy3))
    newtheta = theta +(1.0/6)*(dtheta1+dtheta4+2*(dtheta2+dtheta3))
    
    
    P[0,0]=xnew                                         #Pass New Values
    P[0,1]=ynew
    P[1,0]=Vxnew
    P[1,1]=Vynew
    P[2,0]=newtheta
    P[2,1]=newOmega
    P[3,0]=FL
    P[3,1]=FR
    
    
    return P

print("mass-",m)                              # Systems Parameters
print("weight-",m*9.8)
print("Length-",L)
print("Moment of I-",I)

mass- 500
weight- 4900.0
Length- 10
Moment of I- 4166.666666666666


In [ ]:
from vpython import *
scene = canvas()

scene.width = 350
scene.height = 300
scene.range = 10
scene.title = "2D Drone Simulation \n"
    
sphere(radius=.3)

currentobject=box( length=10, height=1, visible=True )
currentobject.color = color.cyan


def setspeed(s):
    wt.text = '{:1.2f}'.format(s.value)
    wt1.text = '{:1.2f}'.format(s.value)
    
s1 = slider(min=2000, max=2700, value=4900/2, length=220, bind=setspeed, right=15)
s2 = slider(min=2000, max=2700, value=4900/2, length=220, bind=setspeed, right=15)
wt = wtext(text='{:1.2f}'.format(s1.value))
wt1 = wtext(text='{:1.2f}'.format(s2.value))

dt = 0.01
iv=zeros((4,2))
    
while True:
    rate(1/dt)
    iv[3,0]=s1.value
    iv[3,1]=s2.value
    
    itheta=iv[2,0]
    
    iv=RKStep2Ddrone(iv,dt)
    
    dtheta=iv[2,0]-itheta
    
    currentobject.rotate(angle=dtheta, axis=vector(0,0,1))
    
    currentobject.pos=vector(iv[0,0],iv[0,1],0)

    
#iv[0,0]=0 #x
#iv[0,1]=0 #y
#iv[1,0]=0 #Vx
#iv[1,1]=0 #Vy
#iv[2,0]=0 #theta
#iv[2,1]=0 #Omega
#iv[3,0]=0 #FL
#iv[3,1]=0 #FR
    


<IPython.core.display.Javascript object>